# 日本語全文検索の実装
> この章は、`full-text-search-jp.ipynb` を元に作成しています。

## 概要
日本語の全部検索を実装する場合は、以下のような事項について考慮が必要です。

- 多様な文字種: 日本語検索では、ひらがな、カタカナ、漢字、英数字(半角・全角)、特殊記号(①や㌢など)、顔文字など、様々な文字種を取り扱う。
- 表記ゆれへの対応。表記が異なっていても同じ語句として扱う必要がある。
  - 文字種、全角半角、大文字小文字の違いで発生する揺らぎ(あいふぉん、アイフォン、アイフォーン、iphone、i-Phone、iPhone、ｉｐｈｏｎｅ など)
  - 末尾の長音記号(ー)の有無による揺らぎ(コンピューターとコンピュータ)
  - 長音記号とカタカナによる揺らぎ(サラダボールとサラダボウルは同じ単語として処理する必要があるが、バレエとバレーは異なる単語として処理する必要がある)
  - 漢字の踊り字による揺らぎ(明明白白、明々白々)
- 複合語の処理: 複数の語句が結合した複合語は、一つの単語として処理する要件が存在する(山桜、東京タワー、エアバスA300、ホームページ、瀬戸内しまなみ海道 など)
- 類義語の処理類似するキーワードで検索できるようにする必要がある。(正確/的確/明確/確実/確か など)

本ラボでは、OpenSearch で日本語検索実装上の課題にどのように対応するかを解説していきます。

### ラボの構成
本ラボでは、ノートブック環境（EC2 へ Remote Develop 接続した VSCode）および Amazon OpenSearch Serverless を使用します。

<img src="./img/architecture.png" width="512"> 

### 使用するデータセット
本ラボでは、[JGLUE][jglue] 内の FAQ データセットである [JSQuAD][jsquad] を使用します。

[jglue]: https://github.com/yahoojapan/JGLUE
[jsquad]: https://github.com/yahoojapan/JGLUE/tree/main/datasets/jsquad-v1.3

## 事前作業

### パッケージインストール
実行する前に、タブの右上のカーネルの選択を確認してください。

> opensearch と awswrangler のバージョンの依存関係のため、opensearch-py 側を止めています。

In [1]:
!uv add "opensearch-py<3" requests-aws4auth awswrangler[opensearch] ipywidgets python-dotenv

Resolved 62 packages in 161ms                                        
Installed 3 packages in 8ms4.0.14                                
 + ipywidgets==8.1.7
 + jupyterlab-widgets==3.0.15
 + widgetsnbextension==4.0.14


### 環境変数
`.env`ファイルに、以下の環境変数を設定します。

- AOSS_SEARCH_HOST=`AOSS の検索コレクションのエンドポイントのホスト名`
- AOSS_VECTOR_HOST=`AOSS のベクトル検索コレクションのエンドポイントのホスト名`
- AOSS_ROLE_ARN=`AOSS から Bedrock へアクセスするために作成したロールのARN`

設定できたら、以下のコマンドを実行します。

In [1]:
%reload_ext dotenv
%dotenv -o

### インポート

In [2]:
import boto3
import json
import logging

import awswrangler as wr
import pandas as pd
import numpy as np
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

from ipywidgets import interact
import os

### 共通変数のセット

In [5]:
default_region = boto3.Session().region_name
logging.getLogger().setLevel(logging.ERROR)

### OpenSearch Serverless への接続確認
OpenSearch Server のセキュリティ設定により、API リクエストが許可されているかを確認します。

In [6]:
aoss_host = os.getenv("AOSS_SEARCH_HOST")

credentials = boto3.Session().get_credentials()
service_code = "aoss"
auth = AWSV4SignerAuth(credentials=credentials, region=default_region, service=service_code)
opensearch_client = OpenSearch(
    hosts=[{"host": aoss_host, "port": 443}],
    http_compress=True, 
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class = RequestsHttpConnection
)
opensearch_client.cat.indices()

' OPEN jsquad-kuromoji efKH-ZgB_vf32jNhdAKd   67139 0 44.6mb 44.6mb\n OPEN movie-users     cpNv-JgBcgEDf7NLRkzh       1 0  3.6kb  3.6kb\n'

## 日本語検索ウォークスルー
### OpenSearch におけるテキスト処理の全体像

全文検索の対象となるデータは、以下の流れで処理され、転置インデックスに登録されます。

<img src="./img/analyzer.png" width="1024">

以降のセクションでは、各フェーズで登場するコンポーネントの解説と、具体的なコンポーネントの動作を見ていきます。

### Tokenizer
Tokenizer は入力されたテキストを自身のロジックに基づいて分割するコンポーネントです。日本語検索では形態素解析を用いる手法、もしくは n-Gram という N 文字ずつテキストを区切る手法が一般的に用いられます。各手法について実際の挙動を見ていきましょう。

#### N-Gram
N-Gram はテキストから N 文字ずつ取り出してトークン化する手法です。 一文字ずつ取り出すことを uni-gram、二文字ずつ切り取ることを bi-gram、三文字ずつ切り取ることを tri-gram などと呼びます。

ここでは、N-Gram tokenizer で、以下の文字列を 2 文字ずつトークン化した結果を見ていきます。トークンにホワイトスペースや記号が含まれないように、**token_chars** パラメーターで制御を行っています。

"大阪府の関西国際空港(KIX)から東京都の羽田空港(HND)までのフライト時間はおよそ 70 分です"

In [8]:
payload = {
  "text": "大阪府の関西国際空港(KIX)から東京都の羽田空港(HND)までのフライト時間はおよそ 70 分です。",
  "tokenizer": {
    "type": "ngram",
    "min_gram": 2,
    "max_gram": 2,
    "token_chars": ["letter", "digit"]
  }
}

response = opensearch_client.indices.analyze(
  body=payload
)
df_bigram = pd.json_normalize(response["tokens"])
df_bigram

,token,start_offset,end_offset,type,position
0,大阪,0,2,word,0
1,阪府,1,3,word,1
2,府の,2,4,word,2
3,の関,3,5,word,3
4,関西,4,6,word,4
5,西国,5,7,word,5
6,国際,6,8,word,6
7,際空,7,9,word,7
8,空港,8,10,word,8
9,KI,11,13,word,9


上記の例では、文章を 1 文字ずつずらしながら、2 文字のトークンが抽出されたことがわかります。N-Gram は N 文字ずつトークンを抽出することから、未知語に対するヒット率の向上が期待できます。

一方、検索ノイズの増加については考慮が必要です。抽出されたトークンには **"京都"** も含まれているため、`京都`で検索を行った際に無関係の本文章がヒットしてしまいます。

検索ノイズを削減するテクニックとしては以下のようなものが考えられます。

- 複数の N-Gram (bi-gram と tri-gram など)インデックスを併用し、ユーザーが入力した検索キーワードの長さに応じて、アプリケーション側で処理を分岐させる
- 形態素解析と組み合わせる
- トークンフィルターを適用し、"です" や "ます" などの不要な語句(ストップワード)で構成されるトークンを除去する

<div class="alert alert-block alert-warning"> 
<b>N-gram の最小文字数と最大文字数に 2 以上の差がある場合の設定</b>

ngram tokenizer の min_gram および max_gram に 2 以上の差がある場合は、インデックスに [index.max_ngram_diff][index-settings] の設定を追加する必要があります。追加されていない場合、以下のようなエラーが発生します。
</div>

[index-settings]: https://opensearch.org/docs/latest/install-and-configure/configuring-opensearch/index-settings/

In [9]:
payload = {
  "text": "大阪府の関西国際空港(KIX)から東京都の羽田空港(HND)までのフライト時間はおよそ 70 分です。",
  "tokenizer": {
    "type": "ngram",
    "min_gram": 1,
    "max_gram": 3,
  }
}

try:
    response = opensearch_client.indices.analyze(
      body=payload
    )
    df_bigram = pd.json_normalize(response["tokens"])
    df_bigram
except Exception as e:
    print(e)

RequestError(400, 'illegal_argument_exception', 'The difference between max_gram and min_gram in NGram Tokenizer must be less than or equal to: [1] but was [2]. This limit can be set by changing the [index.max_ngram_diff] index level setting.')


#### 形態素解析
形態素解析を用いることで、単語の品詞情報が格納された辞書や文法に基づくトークン分割を行えます。

例えば、吾輩は猫である。 という文章を形態素解析エンジンで処理すると、吾輩 / は / 猫 / で / ある / 。 と自然に分割されたトークンが取得できます。

OpenSearch では、Sudachi もしくは Kuromoji を利用可能ですが、OpenSearch Serverless では、Sudachi は利用できません。以降のセクションでは、Kuromoji の動作を解説していきます。

##### Kuromoji
Kuromoji は Java で実装されたオープンソースの日本語形態素解析ツールです。[atilika][atilika] により開発、Apache Software Foundation に寄贈されており、OpenSearch のベースである Apache Lucene に組み込まれています。Amazon OpenSearch Service および Amazon OpenSearch Serverless では、デフォルトで Kuromoji が利用可能です。

OSS 版の OpenSearch でも、標準の[日本語プラグイン][additional-plugins]として登録されているため、`opensearch-plugin install analysis-kuromoji` コマンドで導入が可能です。

kuromoji_tokenizer は、以下 3 つの分割モードをサポートしています。

- normal: デフォルトのモード。最も長い分割単位でトークンを出力。複合トークンの分割は行わない。
- search: 検索に特化したモード。複合トークンの分割も合わせて行う。
- extended: search の動作に加えて、未知語をユニグラム(1 文字トークン)として出力する

各モードごとの実行結果を見ていきましょう。

[atilika]: https://www.atilika.org/
[additional-plugins]: https://opensearch.org/docs/latest/install-and-configure/additional-plugins/index/

**normal モード**

カッコなどの記号や句読点がトークンに含まれていないのは、kuromoji tokenizer の **discard_punctuation** オプションがデフォルトで `true` になっているためです。記号や句読点をトークンとして含める場合は同設定を `false` にセットします。

In [10]:
payload = {
  "text": "大阪府の関西国際空港(KIX)から東京都の羽田空港(HND)までのフライト時間はおよそ 70 分です。",
  "tokenizer": {
    "type": "kuromoji_tokenizer",
    "mode": "normal",
    "discard_punctuation": True #デフォルト
  }
}

response = opensearch_client.indices.analyze(
  body=payload
)
df_kuromoji_normal = pd.json_normalize(response["tokens"])
df_kuromoji_normal

,token,start_offset,end_offset,type,position
0,大阪,0,2,word,0
1,府,2,3,word,1
2,の,3,4,word,2
3,関西国際空港,4,10,word,3
4,KIX,11,14,word,4
5,から,15,17,word,5
6,東京,17,19,word,6
7,都,19,20,word,7
8,の,20,21,word,8
9,羽田空港,21,25,word,9


**search モード**

In [11]:
payload = {
  "text": "大阪府の関西国際空港(KIX)から東京都の羽田空港(HND)までのフライト時間はおよそ 70 分です",
  "tokenizer": {
    "type": "kuromoji_tokenizer",
    "mode": "search"
  }
}

response = opensearch_client.indices.analyze(
  body=payload
)
df_kuromoji_search = pd.json_normalize(response["tokens"])
df_kuromoji_search

,token,start_offset,end_offset,type,position,positionLength
0,大阪,0,2,word,0,NaN
1,府,2,3,word,1,NaN
2,の,3,4,word,2,NaN
3,関西,4,6,word,3,NaN
4,関西国際空港,4,10,word,3,3.0
5,国際,6,8,word,4,NaN
6,空港,8,10,word,5,NaN
7,KIX,11,14,word,6,NaN
8,から,15,17,word,7,NaN
9,東京,17,19,word,8,NaN


**extended モード**

In [12]:
payload = {
  "text": "大阪府の関西国際空港(KIX)から東京都の羽田空港(HND)までのフライト時間はおよそ 70 分です",
  "tokenizer": {
    "type": "kuromoji_tokenizer",
    "mode": "extended"
  }
}

response = opensearch_client.indices.analyze(
  body=payload
)
df_kuromoji_extended = pd.json_normalize(response["tokens"])
df_kuromoji_extended

,token,start_offset,end_offset,type,position,positionLength
0,大阪,0,2,word,0,NaN
1,府,2,3,word,1,NaN
2,の,3,4,word,2,NaN
3,関西,4,6,word,3,NaN
4,関西国際空港,4,10,word,3,3.0
5,国際,6,8,word,4,NaN
6,空港,8,10,word,5,NaN
7,K,11,12,word,6,NaN
8,I,12,13,word,7,NaN
9,X,13,14,word,8,NaN


**normal/search/extended モードの比較**
3 つのモードを比較します。normal -> search -> extended の順にトークンが増加する様子が分かります。

In [13]:
df_kuromoji_search_and_normal = pd.merge(df_kuromoji_search, df_kuromoji_normal, on=["start_offset", "end_offset"], how="left", suffixes=["_kuromoji_search","_kuromoji_normal"]).drop(["type_kuromoji_search","type_kuromoji_normal","positionLength","position_kuromoji_search", "position_kuromoji_normal"],axis=1).reindex(["start_offset", "end_offset", "token_kuromoji_search", "token_kuromoji_normal"],axis=1).fillna("")
df_kuromoji_extended_and_normal = pd.merge(df_kuromoji_extended, df_kuromoji_normal, on=["start_offset", "end_offset"], how="left", suffixes=["_kuromoji_extended","_kuromoji_normal"]).drop(["type_kuromoji_extended","type_kuromoji_normal","positionLength","position_kuromoji_extended","position_kuromoji_normal"],axis=1).reindex(["start_offset", "end_offset", "token_kuromoji_extended", "token_kuromoji_normal"],axis=1)
df_kuromoji = pd.merge(df_kuromoji_extended_and_normal, df_kuromoji_search_and_normal, on=["start_offset"], how="left").drop(["token_kuromoji_normal_x"],axis=1).rename(columns={"token_kuromoji_normal_y": "token_kuromoji_normal"}).reindex(["start_offset", "token_kuromoji_extended", "token_kuromoji_search", "token_kuromoji_normal"],axis=1).fillna("")
df_kuromoji

,start_offset,token_kuromoji_extended,token_kuromoji_search,token_kuromoji_normal
0,0,大阪,大阪,大阪
1,2,府,府,府
2,3,の,の,の
3,4,関西,関西,
4,4,関西,関西国際空港,関西国際空港
5,4,関西国際空港,関西,
6,4,関西国際空港,関西国際空港,関西国際空港
7,6,国際,国際,
8,8,空港,空港,
9,11,K,KIX,KIX


なお、search もしくは extended モードで、分割前の複合語を破棄する場合は、**discard_compound_token** に `true` をセットします。以下は search モードにおける **discard_compound_token** パラメーターによる結果の違いです。

In [14]:
payload = {
  "text": "大阪府の関西国際空港(KIX)から東京都の羽田空港(HND)までのフライト時間はおよそ 70 分です",
  "tokenizer": {
    "type": "kuromoji_tokenizer",
    "mode": "search",
    "discard_compound_token": True
  }
}

response = opensearch_client.indices.analyze(
  body=payload
)
df_kuromoji_search_discard_compound_token = pd.json_normalize(response["tokens"])
df_kuromoji_search_results = pd.merge(df_kuromoji_search, df_kuromoji_search_discard_compound_token, on=["start_offset", "end_offset"], how="left", suffixes=["_without_discard_compound_token","_with_discard_compound_token"]).drop(["type_without_discard_compound_token","type_with_discard_compound_token","positionLength","position_without_discard_compound_token", "position_with_discard_compound_token"],axis=1).reindex(["start_offset", "end_offset", "token_without_discard_compound_token", "token_with_discard_compound_token"],axis=1).fillna("")
df_kuromoji_search_results

,start_offset,end_offset,token_without_discard_compound_token,token_with_discard_compound_token
0,0,2,大阪,大阪
1,2,3,府,府
2,3,4,の,の
3,4,6,関西,関西
4,4,10,関西国際空港,
5,6,8,国際,国際
6,8,10,空港,空港
7,11,14,KIX,KIX
8,15,17,から,から
9,17,19,東京,東京


### Character Filter
Tokenizer に渡す前段での正規化を担当するコンポーネントです。不要な文字の除去や半角・全角を揃えるなどの正規化処理を行うことで、表記ゆれによる検索精度の低下を防ぎます。

Character Filter には踊り字の置き換えといった、トークン分割自体の精度向上に寄与するものもあります。

#### ICU normalization character filter

ICU normalization character filter は、文字列の正規化処理を行うフィルターです。以下のような表記ゆれを補正可能です。

| 変換内容 | 変換例(前) |変換例(後)|
| ---- | ---- | ---- |
| 大文字 -> 小文字 | OpenSearch | opensearch |
| 全角英数字・記号 -> 半角英数字・記号 | oｐeｎ＿sｅaｒcｈ | open_search |
| 半角カナ -> 全角カナ | ｵｰﾌﾟﾝｿｰｽ	| オープンソース |
| 数字記号 -> 半角数字 | ① | 1 |
| 単位記号 -> 全角カナ | ㍍ | メートル |

以下の例では、様々な種類の文字が混在する文字列の正規化を行っています。

In [15]:
payload = {
  "text": "OｐeｎsｅaｒCｈは①⓪⓪㌫ｵｰﾌﾟンｿｰｽの検索／分析スイートです",
  "tokenizer": {
    "type": "kuromoji_tokenizer"
  }
}
response = opensearch_client.indices.analyze(
  body=payload
)
df_sudachi = pd.json_normalize(response["tokens"])

payload = {
  "text": "OｐeｎsｅaｒCｈは①⓪⓪㌫ｵｰﾌﾟンｿｰｽの検索／分析スイートです",
  "tokenizer": {
    "type": "kuromoji_tokenizer"
  },
  "char_filter": ["icu_normalizer"]
}
response = opensearch_client.indices.analyze(
  body=payload
)
df_sudachi_normalized = pd.json_normalize(response["tokens"])

pd.merge(df_sudachi, df_sudachi_normalized, on=["start_offset","end_offset"], how="outer").rename(columns={"token_x": "token", "token_y": "token_normalized"}).reindex(["start_offset", "end_offset", "token", "token_normalized"],axis=1).fillna("")


,start_offset,end_offset,token,token_normalized
0,0,10,OｐeｎsｅaｒCｈ,opensearch
1,10,11,は,は
2,11,14,①⓪⓪,100
3,14,15,,パーセント
4,15,20,,オープン
5,15,23,ｵｰﾌﾟンｿｰｽ,
6,20,23,,ソース
7,23,24,の,の
8,24,26,検索,検索
9,27,29,分析,分析


#### kuromoji_iteration_mark character filter
kuromoji_iteration_mark は、踊り字(々, ゝ, ヽ)を直前の文字で置き換える機能を提供します。<br>
踊り字を変換せずにそのままトークン分割を行った場合、以下のような問題が発生します

- トークン分割時に踊り字だけがインデクシングされてしまう
- 踊り字を含むキーワードで検索を行った際に、踊り字を含むすべてのキーワードがヒットしてしまう
- 文字列の分割箇所がおかしくなる

例えば、**こゝろ** や **つゝむ** をそのまま Kuromoji Tokenizer で処理すると、ゝ が一つのトークンとして抽出されます。このままの状態でインデックスにデータが格納された場合、`こゝろ` で検索を行うと、**つゝむ** もヒットしてしまいます。

また、学問のすゝめ については、学問/の/すゝ/め と不自然な位置で区切られてしまいます。

In [16]:
payload = {
  "text": "こゝろ",
  "tokenizer": {
    "type": "kuromoji_tokenizer"
  }
}
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,こ,0,1,word,0
1,ゝ,1,2,word,1
2,ろ,2,3,word,2


In [17]:
payload = {
  "text": "つゝむ",
  "tokenizer": {
    "type": "kuromoji_tokenizer"
  }
}
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,つ,0,1,word,0
1,ゝ,1,2,word,1
2,む,2,3,word,2


In [18]:
payload = {
  "text": "学問のすゝめ",
  "tokenizer": {
    "type": "kuromoji_tokenizer"
  }
}
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,学問,0,2,word,0
1,の,2,3,word,1
2,すゝ,3,5,word,2
3,め,5,6,word,3


`kuromoji_iteration_mark` を利用することで、踊り字がひとつ前の文字に置き換えられ、トークンが正しく抽出されるようになります

In [19]:
payload = {
  "text": "学問のすゝめ",
  "tokenizer": {
    "type": "kuromoji_tokenizer"
  },
  "char_filter": ["kuromoji_iteration_mark"]
}
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,学問,0,2,word,0
1,の,2,3,word,1
2,すすめ,3,6,word,2


### Token Filter
Token Filter は Tokenizer によって分割・抽出されたトークンに対する処理を行います。検索ノイズの増加に影響するストップワードや特定の品詞の除去、ステミングや表記ゆれの補正など、検索精度を向上するうえで欠かせない処理が提供されています。<br>
以降、主要な Token Filter について解説していきます。

なお、Token Filter の中には、品詞分類などを手掛かりとして処理を行うものが存在します。こうした処理は、同じプラグイン(Kuromoji、Sudachi)でトークナイズされていることが前提となるため、Kuromoji で生成されたトークンを Sudachi のトークンフィルタで処理できない場合があります。

#### 原形への置き換え
変化形を原形に置き換えてインデックスへの格納・検索を行うことで、食べる と 食べた といった形の違いによる検索ヒット率の低下を防ぎます。
Kuromoji でトークン分割を行った場合は **kuromoji_baseform** Token Filter を使用します。

In [20]:
payload = {
  "tokenizer": "kuromoji_tokenizer",
  "text": "寿司を食べた。美味しかったな"
}
response = opensearch_client.indices.analyze(
  body=payload
)
df_kuromoji = pd.json_normalize(response["tokens"])

payload = {
  "tokenizer": "kuromoji_tokenizer",
  "filter": ["kuromoji_baseform"],
  "text": "寿司を食べた。美味しかったな"
}
response = opensearch_client.indices.analyze(
  body=payload
)
df_kuromoji_baseform = pd.json_normalize(response["tokens"])

pd.merge(df_kuromoji_baseform, df_kuromoji, on=["start_offset","end_offset"], how="outer").rename(columns={"token_x": "token_baseform", "token_y": "token"}).reindex(["start_offset", "end_offset", "token", "token_baseform"],axis=1).fillna("")


,start_offset,end_offset,token,token_baseform
0,0,2,寿司,寿司
1,2,3,を,を
2,3,5,食べ,食べる
3,5,6,た,た
4,7,12,美味しかっ,美味しい
5,12,13,た,た
6,13,14,な,な


#### 品詞分類によるトークン除去
トークナイザーにより抽出されたトークンには品詞の情報が付与されています。品詞分類を元に、助詞や接続詞などの検索ノイズになりうるトークンを削除します。

In [21]:
payload = {
  "tokenizer": "kuromoji_tokenizer",
  "filter": ["kuromoji_baseform"],
  "text": "寿司を食べた。美味しかったな"
}
response = opensearch_client.indices.analyze(
  body=payload
)
df_kuromoji_baseform = pd.json_normalize(response["tokens"])

payload = {
  "tokenizer": "kuromoji_tokenizer",
  "filter": [
    "kuromoji_baseform",
    {
        "type": "kuromoji_part_of_speech",
        "stoptags": [
          "助詞-格助詞-一般",
          "助動詞",
          "助詞-終助詞"
        ]
    }
  ],
  "text": "寿司を食べた。美味しかったな"
}
response = opensearch_client.indices.analyze(
  body=payload
)
df_kuromoji_baseform_part_of_speech = pd.json_normalize(response["tokens"])

pd.merge(df_kuromoji_baseform, df_kuromoji_baseform_part_of_speech, on=["start_offset","end_offset"], how="outer").rename(columns={"token_x": "token_baseform", "token_y": "token_baseform_part_of_speech"}).reindex(["start_offset", "end_offset", "token_baseform", "token_baseform_part_of_speech"],axis=1).fillna("")


,start_offset,end_offset,token_baseform,token_baseform_part_of_speech
0,0,2,寿司,寿司
1,2,3,を,
2,3,5,食べる,食べる
3,5,6,た,
4,7,12,美味しい,美味しい
5,12,13,た,
6,13,14,な,


#### ストップワードの除去
日本語における "てにをは" など、検索において重要ではない語句をストップワードと呼びます。<br>
ストップワードがインデックスに格納されると検索性が低下するため、一般的にはインデックスに格納されないよう除去します。品詞単位の除去に似ていますが、ストップワードの除去は品詞の分類による判断ではなく、ストップワードリストを元に判断します。

In [22]:
payload = {
  "tokenizer": "kuromoji_tokenizer",
  "filter": [
    "kuromoji_baseform"
  ],
  "text": "寿司を食べた。美味しかったな"
}
response = opensearch_client.indices.analyze(
  body=payload
)
df_kuromoji = pd.json_normalize(response["tokens"])

payload = {
  "tokenizer": "kuromoji_tokenizer",
  "filter": [
    "kuromoji_baseform",
    {
      "type": "ja_stop",
      "stopwords": ["_japanese_","寿司"]
    }
  ],
  "text": "寿司を食べた。美味しかったな"
}
response = opensearch_client.indices.analyze(
  body=payload
)
df_kuromoji_sudachi_ja_stop = pd.json_normalize(response["tokens"])

pd.merge(df_kuromoji, df_kuromoji_sudachi_ja_stop, on=["start_offset","end_offset"], how="outer").rename(columns={"token_x": "token", "token_y": "token_ja_stop"}).reindex(["start_offset", "end_offset", "token", "token_ja_stop"],axis=1).fillna("")


,start_offset,end_offset,token,token_ja_stop
0,0,2,寿司,
1,2,3,を,
2,3,5,食べる,食べる
3,5,6,た,
4,7,12,美味しい,美味しい
5,12,13,た,
6,13,14,な,


#### 類義語
OpenSearch では類義語を同じ語句として取り扱うことで検索精度を向上させます。

例えば、"パイン"、"パイナップル" など、同じものを指していても、表記が異なれば異なるキーワードとして扱われます。以下は実際の動作例です。

In [23]:
payload = {
  "tokenizer": "kuromoji_tokenizer",
  "text": ["パイン", "パイナップル"]
} 
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,パイン,0,3,word,0
1,パイナップル,4,10,word,101


シノニムを設定することで、インデクシング時および検索時にテキストの類義語を展開することができます。

In [24]:
payload = {
  "tokenizer": "kuromoji_tokenizer",
  "filter": [
    {
      "type": "synonym",
      "lenient": False,
      "synonyms": [ "パイン=> パイナップル" ]
    }
  ],
  "text": ["パインゼリー", "パイナップルアイス"]
} 
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,パイナップル,0,3,SYNONYM,0
1,ゼリー,3,6,word,1
2,パイナップル,7,13,word,102
3,アイス,13,16,word,103


_analyze API の実行結果で type が SYNONYM となっているものは、シノニムの定義により展開・出力されたトークンであることを表します。上記の例でパインがパイナップルに変化したのは、シノニム設定時に、矢印 (=>) で展開方向を抑制しているためです。 矢印 (=>) で展開方向を抑制したことで、パイン は パイナップルに変換されてからインデックスに格納されます

一方、矢印を記載せずにカンマで区切った場合、シノニムは相互展開されます。以下は展開例です。

In [25]:
payload = {
  "tokenizer": "kuromoji_tokenizer",
  "filter": [
    {
      "type": "synonym",
      "lenient": False,
      "synonyms": [ "パイン,パイナップル" ]
    }
  ],
  "text": ["パインゼリー", "パイナップルアイス"]
} 
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,パイン,0,3,word,0
1,パイナップル,0,3,SYNONYM,0
2,ゼリー,3,6,word,1
3,パイナップル,7,13,word,102
4,パイン,7,13,SYNONYM,102
5,アイス,13,16,word,103


#### カナおよびローマ字読みへの変換
トークンをカナ表記、ローマ字表記に変換することで検索ワードの揺らぎを補正することが可能です。

Sudachi と Kuromoji それぞれで固有の readingform filter を使用する必要があります。kuromoji_tokenizer に対しては kuromoji_readingform を使用します。

use_romaji オプションを true にするとローマ字に、false にするとカタカナに変換されます。

In [26]:
payload = {
  "tokenizer": "kuromoji_tokenizer",
  "filter": [
    {
      "type": "kuromoji_readingform",
      "use_romaji": True
    },
  ],
  "text": ["いか", "烏賊", "イカ"]
} 
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,ika,0,2,word,0
1,ika,3,5,word,101
2,ika,6,8,word,202


変換の精度は辞書に依存します。例えば、"紅まどんな(べにまどんな)" は Kuromoji のデフォルトシステム辞書に登録されていないため、トークン分割された上に "べに" ではなく "あか" と読まれてしまいます。
カスタム辞書に読み仮名を含めて登録することで対処可能です。

In [27]:
payload = {
  "tokenizer": "kuromoji_tokenizer",
  "filter": [
    {
      "type": "kuromoji_readingform",
      "use_romaji": False
    },
  ],
  "text": ["紅まどんな"]
} 
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,アカ,0,1,word,0
1,マ,1,2,word,1
2,ドンナ,2,5,word,2


もう一つの注意点として、同音異字も同じ文字に変換されます。これは検索ノイズの増加につながる可能性があります

In [28]:
payload = {
  "tokenizer": "kuromoji_tokenizer",
  "filter": [
    {
      "type": "kuromoji_readingform",
      "use_romaji": False
    },
  ],
  "text": ["感情", "勘定", "環状"]
} 
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,カンジョウ,0,2,word,0
1,カンジョウ,3,5,word,101
2,カンジョウ,6,8,word,202


#### その他の正規化機能
その他、各形態素解析器固有の機能について解説していきます。

##### 長音記号のステミング (Kuromoji)
Kuromoji kuromoji_stemmer と呼ばれるトークン末尾の長音記号(ー)を削除する機能を提供します。minimum_length オプションで、長音記号を削除するトークンの最小文字数を指定することが可能です。

* minimum_length オプションで指定した文字長未満のトークンは末尾の長音記号削除は行われません。デフォルト値は 4 です。この数値は以前の JISZ8301 にて、3音以上の言葉については語尾に長音符号を付けない、2音以下の言葉については語尾に調音符号を付与するというものに由来していると考えられます。2024 年現在の JISZ8301 ではこの基準は削除されています。
* 本 Token Filter は全角カナのみが対象となるため、半角カナや全角かなに適用するためには、icu_normalizer による半角カナ->全角カナの置き換えや、kuromoji_readingform による全角かな->全角カナへの置き換えが必要です。


In [29]:
payload = {
  "tokenizer": "kuromoji_tokenizer",
  "filter": [
    {
      "type": "kuromoji_stemmer",
      "minimum_length": 4 #default
    }
  ],
  "text": ["コピー", "サーバー"]
}
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,コピー,0,3,word,0
1,サーバ,4,8,word,101


詳細については、[内閣告示・内閣訓令 「外来語の表記　留意事項その2(細則的な事項)」][gairai]や、JTCA の[「TC 関連ガイドライン」][tc_guide]をご覧ください。

[gairai]: https://www.bunka.go.jp/kokugo_nihongo/sisaku/joho/joho/kijun/naikaku/gairai/honbun06.html
[tc_guide]: https://jtca.org/useful/tc_guide/

語末の長音記号の有無による表記ゆれを解消できる本 Token Filter ですが、長音記号を削除することで元々の単語の意味が変わってしまう副作用には注意が必要です。

例えば、コーラー(caller) 末尾の長音記号を削除した場合、生成されるトークンは コーラ(Cola) となり語句の意味自体が変わってしまいます。

このような問題を抑制するために minimum_length 設定があります。デフォルト値の 4 を使用した場合、以下のようなケースを防止可能です。

- エコー(echo) -> エコ(eco)
- エラー(error) -> エラ(era)
- カバー(cover) -> カバ

##### アラビア数字への置き換え (Kuromoji)
Kuromoji は kuromoji_number と呼ばれる、漢数字をアラビア数字に置換する機能を提供します。置換対象の漢数字は Lucene の [JapaneseNumberFilter.java](https://github.com/apache/lucene/blob/main/lucene/analysis/kuromoji/src/java/org/apache/lucene/analysis/ja/JapaneseNumberFilter.java) より確認可能です。

対応している単位は垓(10 の 20 乗) までです。

アラビア数字への置き換えは、Tokenizer により分割されたトークンが漢数字で構成された文字列のみが対象となります。

In [30]:
payload = {
  "tokenizer": "kuromoji_tokenizer",
  "filter": [
    {
      "type": "kuromoji_number"
    }
  ],
  "text": ["千垓千一",  "二千,五百十円です", "千載一遇"]
}
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position,positionLength
0,100000000000000000001001,0,4,word,0,NaN
1,2510,5,11,word,101,NaN
2,円,11,12,word,102,NaN
3,です,12,14,word,103,NaN
4,千載,15,17,word,204,NaN
5,千載一遇,15,19,word,204,3.0
6,一,17,18,word,205,NaN
7,遇,18,19,word,206,NaN


## 日本語検索の実行
サンプルインデックスにデータをロードし、いくつかの日本語検索を実行していきます。

### サンプルデータの準備

In [31]:
%%time
dataset_dir = "./dataset/jsquad"
%mkdir -p $dataset_dir
!curl -L -s -o $dataset_dir/valid.json https://github.com/yahoojapan/JGLUE/raw/main/datasets/jsquad-v1.3/valid-v1.3.json 
!curl -L -s -o $dataset_dir/train.json https://github.com/yahoojapan/JGLUE/raw/main/datasets/jsquad-v1.3/train-v1.3.json 

CPU times: user 40.9 ms, sys: 1.01 ms, total: 41.9 ms
Wall time: 2.67 s


In [32]:
%%time
import pandas as pd
import json

def squad_json_to_dataframe(input_file_path, record_path=["data", "paragraphs", "qas", "answers"]):
    file = json.loads(open(input_file_path).read())
    m = pd.json_normalize(file, record_path[:-1])
    r = pd.json_normalize(file, record_path[:-2])

    idx = np.repeat(r["context"].values, r.qas.str.len())
    m["context"] = idx
    m["answers"] = m["answers"]
    m["answers"] = m["answers"].apply(lambda x: np.unique(pd.json_normalize(x)["text"].to_list()))
    return m[["id", "question", "context", "answers"]]

valid_filename = f"{dataset_dir}/valid.json"
valid_df = squad_json_to_dataframe(valid_filename)

train_filename = f"{dataset_dir}/train.json"
train_df = squad_json_to_dataframe(train_filename)

CPU times: user 8.44 s, sys: 126 ms, total: 8.56 s
Wall time: 8.57 s


### サンプルデータの確認
サンプルデータは日本語の FAQ データセットです。<br>質問文フィールドの question、回答の answers、説明文の context フィールド、問題 ID である id フィールドから構成されています。

In [33]:
valid_df

,id,question,context,answers
0,a10336p0q0,日本で梅雨がないのは北海道とどこか。,梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の...,"[小笠原諸島, 小笠原諸島を除く日本]"
1,a10336p0q1,梅雨とは何季の一種か?,梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の...,[雨季]
2,a10336p0q2,梅雨は、世界的にどのあたりで見られる気象ですか？,梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の...,"[東アジア, 東アジアの広範囲]"
3,a10336p0q3,梅雨がみられるのはどの期間？,梅雨 [SEP] 梅雨（つゆ、ばいう）は、北海道と小笠原諸島を除く日本、朝鮮半島南部、中国の...,"[5月から7月, 5月から7月にかけて]"
4,a10336p1q0,入梅は何の目安の時期か？,梅雨 [SEP] 梅雨の時期が始まることを梅雨入りや入梅（にゅうばい）といい、社会通念上・気...,"[春の終わりであるとともに夏の始まり（初夏）, 田植えの時期, 田植えの時期の目安]"
...,...,...,...,...
4437,a95156p5q3,国際銀行間通信協会ならびに国際決済機関の何と何も企業体である,多国籍企業 [SEP] 国際銀行間通信協会ならびに国際決済機関のクリアストリームとユーロクリ...,[クリアストリームとユーロクリア]
4438,a95156p6q0,ゼネコンはどの国特有の形態か,多国籍企業 [SEP] ゼネコンは日本特有の形態。セメントメジャーにラファージュホルシムやイ...,[日本]
4439,a95156p6q1,多国籍企業においてゼネコンはどこの国特有の形態であるか？,多国籍企業 [SEP] ゼネコンは日本特有の形態。セメントメジャーにラファージュホルシムやイ...,[日本]
4440,a95156p6q2,多国籍企業を一つ挙げよ,多国籍企業 [SEP] ゼネコンは日本特有の形態。セメントメジャーにラファージュホルシムやイ...,"[イタルチェメンティ, ラファージュホルシム]"


In [34]:
train_df

,id,question,context,answers
0,a1000888p0q0,新たに語（単語）を造ることや、既存の語を組み合わせて新たな意味の語を造ること,造語 [SEP] 造語（ぞうご）は、新たに語（単語）を造ることや、既存の語を組み合わせて新た...,[造語]
1,a1000888p0q1,新たに造られた語のことを新語または何という？,造語 [SEP] 造語（ぞうご）は、新たに語（単語）を造ることや、既存の語を組み合わせて新た...,[新造語]
2,a1000888p0q2,たに語（単語）を造ることや、既存の語を組み合わせて新たな意味の語を造ること、また、そうして造...,造語 [SEP] 造語（ぞうご）は、新たに語（単語）を造ることや、既存の語を組み合わせて新た...,[造語]
3,a1000888p0q3,新たに語を造ることや、既存の語を組み合わせて新たな意味の語を造ることを何という？,造語 [SEP] 造語（ぞうご）は、新たに語（単語）を造ることや、既存の語を組み合わせて新た...,[造語]
4,a1000888p0q4,既存の語を組み合わせたりして新しく単語を造ることを何と言う？,造語 [SEP] 造語（ぞうご）は、新たに語（単語）を造ることや、既存の語を組み合わせて新た...,[造語]
...,...,...,...,...
62692,a99943p9q0,ストラングラーズは、どんな車で各地を回っていたか？,パンク・ロック [SEP] 他に、ザ・ジャムがネオ・モッズ・ムーブメントを巻き起こし、UKチ...,[アイスクリーム販売用のバン]
62693,a99943p9q1,ザ・ジャムが解散したのはいつか？,パンク・ロック [SEP] 他に、ザ・ジャムがネオ・モッズ・ムーブメントを巻き起こし、UKチ...,[1982年]
62694,a99943p9q2,ストラングラーズは、イギリス国内を何で移動してライヴを行った？,パンク・ロック [SEP] 他に、ザ・ジャムがネオ・モッズ・ムーブメントを巻き起こし、UKチ...,[アイスクリーム販売用のバン]
62695,a99943p9q3,ザ・ジャムが解散したのは何年か。,パンク・ロック [SEP] 他に、ザ・ジャムがネオ・モッズ・ムーブメントを巻き起こし、UKチ...,[1982年]


### インデックス作成

In [35]:
index_name = "jsquad-kuromoji"

payload = {
    "mappings": {
        "properties": {
            "id": {"type": "keyword"},
            "question": {"type": "text", "analyzer": "custom_kuromoji_analyzer"},
            "context": {"type": "text", "analyzer": "custom_kuromoji_analyzer"},
            "answers": {"type": "text", "analyzer": "custom_kuromoji_analyzer"},
        }
    },
    "settings": {
        "index.number_of_shards": 1,
        "index.number_of_replicas": 0,
        "analysis": {
            "analyzer": {
                "custom_kuromoji_analyzer": {
                    "char_filter": ["icu_normalizer"],
                    "filter": [
                        "custom_kuromoji_part_of_speech",
                    ],
                    "tokenizer": "kuromoji_tokenizer",
                    "type": "custom",
                }
            },
            "filter": {
                "custom_kuromoji_part_of_speech": {
                    "type": "kuromoji_part_of_speech",
                    "stoptags": ["感動詞,フィラー","接頭辞","代名詞","副詞","助詞","助動詞","動詞,一般,*,*,*,終止形-一般","名詞,普通名詞,副詞可能"]
                }
            },
        },
    },
}

try:
    # 既に同名のインデックスが存在する場合、いったん削除を行う
    print("# delete index")
    response = opensearch_client.indices.delete(index=index_name)
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)

# インデックスの作成を行う
print("# create index")
response = opensearch_client.indices.create(index=index_name, body=payload)
print(json.dumps(response, indent=2))

# delete index
NotFoundError(404, 'index_not_found_exception', 'no such index [jsquad-kuromoji]')
# create index
{
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "jsquad-kuromoji"
}


### ドキュメントのロード
ドキュメントのロードを行います。<br>ドキュメントのロードは "OpenSearch の基本概念・基本操作の理解" でも解説した通り bulk API を使用することで効率よく進められますが、データ処理フレームワークを利用することでより簡単にデータを取り込むことも可能です。本ワークショップでは、[AWS SDK for Pandas][aws-sdk-pandas] を使用したデータ取り込みを行います。

[aws-sdk-pandas]: https://github.com/aws/aws-sdk-pandas


In [36]:
%%time
index_name = "jsquad-kuromoji"

response = wr.opensearch.index_df(
    client=opensearch_client,
    df=pd.concat([train_df, valid_df]),
    use_threads=True,
    id_keys=["id"],
    index=index_name,
    bulk_size=1000,
    refresh=False
)

CPU times: user 6.08 s, sys: 46.9 ms, total: 6.13 s
Wall time: 29.6 s


response["success"] の値が DataFrame の件数と一致しているかを確認します。<br>True が表示される場合は全件登録に成功していると判断できます。

In [37]:
response["success"] == pd.concat([train_df, valid_df]).id.count()

np.True_

### インタラクティブな検索
以降は時間の許す限り、自由に検索クエリを実行してみましょう

- query テキストボックスの内容を書き換えることで、検索クエリを変更することが可能です
- question、context、answers のチェックボックスを ON/OFF で切り替えることで、フィールド単位で検索可否を調整可能です。
- 具体的にどの個所にヒットしたかは、highlight.<field-name> のカラムから確認可能です。

In [ ]:
def search(index_name, query, question, context, answers):
    fields = []
    if question:
        fields.append("question")
    if context:
        fields.append("context")
    if answers:
        fields.append("answers")
    payload = {
      "query": {
        "multi_match": {
          "query": query,
          "fields": fields,
          "operator": "and"
        }
      },
      "highlight": {
        "fields": {
          "*" : {}
        }
      },
      "_source": False,
      "fields": fields
    }
    response = opensearch_client.search(
        index=index_name,
        body=payload
    )
    return pd.json_normalize(response["hits"]["hits"])

index_name = "jsquad-kuromoji"
query = "シュミレーション 言語"

# テキストボックス
interact(search, index_name=index_name, query=query, question=True, context=True, answers=True)

interactive(children=(Text(value='jsquad-kuromoji', description='index_name'), Text(value='シュミレーション 言語', descr…

<function __main__.search(index_name, query, question, context, answers)>

# Kuromoji ユーザー辞書のカスタマイズによる日本語検索の精度改善
> この章は、`kutomoji-user-dictionary.ipynb` を元に作成しています。

## 概要
形態素解析を使用して文章のトークン化を行う場合、辞書が単語認識のベースとなります。したがって、トークン分割の結果がユーザーから見て直感的かどうかは、辞書に依存します。

日本語検索プラグインである Kuromoji では、デフォルトで備えている標準辞書に加えて、ユーザー辞書を追加することでトークン分割を適正化することができます。

本ラボでは、Kuromoji の標準辞書のカバー範囲と、ユーザー辞書によるトークン分割の改善を実際に行っていきます。

## Kuromoji 標準辞書

Kuromoji 標準辞書に登録されている単語は [mecab-ipadic-2.7.0-20070801.tar.gz](http://atilika.com/releases/mecab-ipadic/mecab-ipadic-2.7.0-20070801.tar.gz) 内のファイルから確認することができます。以下はファイルのリストです。

| ファイル名                 | 分類                    | 例                                   |
| -------------------------- | ----------------------- | ------------------------------------ |
| Adj.csv.utf8.txt           | 形容詞                  | 軽い、何気無い、優しい               |
| Adnominal.csv.utf8.txt     | 連体詞                  | 確固たる、いわゆる、おかしな         |
| Adverb.csv.utf8.txt        | 副詞                    | ぜったいに、多少、なにしろ           |
| Auxil.csv.utf8.txt         | 助動詞                  | です、ます、らしく、ある             |
| Conjunction.csv.utf8.txt   | 接続詞                  | なので、でも、なお、なら             |
| Filler.csv.utf8.txt        | フィラーワード          | あー、えー、うん、まあ               |
| Interjection.csv.utf8.txt  | 感動詞(感嘆詞)          | わあ、へー、おはよう                 |
| Noun.adjv.csv.utf8.txt     | 名詞(形容動詞語幹)      | きらびやか、温厚、人一倍             |
| Noun.adverbal.csv.utf8.txt | 名詞(副詞可能)          | すべて、全員、近頃                   |
| Noun.csv.utf8.txt          | 名詞(一般)              | 氏名、コスト、足ぶみ、わたぼうし     |
| Noun.demonst.csv.utf8.txt  | 名詞(代名詞)            | 私、君、あれ、これ、それ             |
| Noun.nai.csv.utf8.txt      | 名詞("ない" 形容詞語幹) | 申しわけ、しょうが、他愛             |
| Noun.name.csv.utf8.txt     | 名詞(固有名詞/人名)     | ノーベル、蘇我蝦夷、長崎、頼朝       |
| Noun.number.csv.utf8.txt   | 名詞(数)                | 百、１(全角)、ゼロ、ひと             |
| Noun.org.csv.utf8.txt      | 名詞(固有名詞/組織)     | 国会図書館、最高裁判所、造幣局       |
| Noun.others.csv.utf8.txt   | 名詞(非自立)            | かぎり、はず、矢先、つもり           |
| Noun.place.csv.utf8.txt    | 名詞(固有名詞/地域)     | 関東、東京、目黒                     |
| Noun.proper.csv.utf8.txt   | 名詞(固有名詞/一般)     | アマゾン川、幕張メッセ、金毘羅山     |
| Noun.verbal.csv.utf8.txt   | 名詞(サ辺接続)          | 改善、感謝、リスクヘッジ、こざっぱり |
| Others.csv.utf8.txt        | その他                  | よ、ァ                               |
| Postp-col.csv.utf8.txt     | 助詞(格助詞)            | にあたります、を通じて               |
| Postp.csv.utf8.txt         | 助詞(特殊)              | て、に、を、は、けども、ながら       |
| Prefix.csv.utf8.txt        | 接頭詞                  | 真、大、小、今                       |
| Suffix.csv.utf8.txt        | 名詞(接尾/助数詞)       | 人、係、メートル                     |
| Symbol.csv.utf8.txt        | 記号                    | ￥、Σ、●、〒                       |
| Verb.csv.utf8.txt          | 動詞                    | 探し出す、学ぶ、ぬかるむ             | 

辞書のエントリファイルの書式は以下のようになっています。

`表層形,左文脈ID,右文脈ID,コスト,品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音`

形容詞や動詞は、活用形ごとに辞書内にエントリが存在し、共通の原形が割り当てられています。活用形ごとに原形を伴って辞書に情報が登録されていることで、活用形の違いによる検索ヒット率の低下を、後段で解説する正規化処理で防ぐことができます。以下は一部ファイルの抜粋です。

### Adj.csv.utf8.txt
```csv
あたたかい,19,19,6948,形容詞,自立,*,*,形容詞・アウオ段,基本形,あたたかい,アタタカイ,アタタカイ
あたたかし,23,23,6953,形容詞,自立,*,*,形容詞・アウオ段,文語基本形,あたたかい,アタタカシ,アタタカシ
あたたかから,27,27,6953,形容詞,自立,*,*,形容詞・アウオ段,未然ヌ接続,あたたかい,アタタカカラ,アタタカカラ
あたたかかろ,25,25,6953,形容詞,自立,*,*,形容詞・アウオ段,未然ウ接続,あたたかい,アタタカカロ,アタタカカロ
あたたかかっ,33,33,6952,形容詞,自立,*,*,形容詞・アウオ段,連用タ接続,あたたかい,アタタカカッ,アタタカカッ
あたたかく,35,35,6952,形容詞,自立,*,*,形容詞・アウオ段,連用テ接続,あたたかい,アタタカク,アタタカク
```

### Verb.csv.utf8.txt
```csv
すみわたる,772,772,9279,動詞,自立,*,*,五段・ラ行,基本形,すみわたる,スミワタル,スミワタル
すみわたら,780,780,9279,動詞,自立,*,*,五段・ラ行,未然形,すみわたる,スミワタラ,スミワタラ
すみわたん,782,782,9279,動詞,自立,*,*,五段・ラ行,未然特殊,すみわたる,スミワタン,スミワタン
すみわたろ,778,778,9279,動詞,自立,*,*,五段・ラ行,未然ウ接続,すみわたる,スミワタロ,スミワタロ
すみわたり,788,788,9279,動詞,自立,*,*,五段・ラ行,連用形,すみわたる,スミワタリ,スミワタリ
すみわたっ,786,786,9279,動詞,自立,*,*,五段・ラ行,連用タ接続,すみわたる,スミワタッ,スミワタッ
```

### Noun.verbal.csv.utf8.txt

<div class="alert alert-block alert-info"> 
<b>名詞は活用形を持たないため、原形のみが登録されています。</b>
</div>

```csv
確言,1283,1283,4467,名詞,サ変接続,*,*,*,*,確言,カクゲン,カクゲン
行脚,1283,1283,4466,名詞,サ変接続,*,*,*,*,行脚,アンギャ,アンギャ
微笑,1283,1283,4087,名詞,サ変接続,*,*,*,*,微笑,ビショウ,ビショー
ミート,1283,1283,4426,名詞,サ変接続,*,*,*,*,ミート,ミート,ミート
含有,1283,1283,4467,名詞,サ変接続,*,*,*,*,含有,ガンユウ,ガンユー
```

## ユーザー辞書
エンジンに組み込まれているデフォルトの辞書は全ての固有名詞をカバーしないため、ユーザー辞書に語句を登録することでトークンの抽出が意図したとおりに行われるようになります。

### デフォルト辞書が機能しない例

標準の Kuromoji Tokenizer では、**[紅まどんな(べにまどんな)](https://ja.wikipedia.org/wiki/%E6%84%9B%E5%AA%9B%E6%9E%9C%E8%A9%A6%E7%AC%AC28%E5%8F%B7)** というキーワードは **紅/ま/どんな** と分割されます。

以下は _analyze API の実行例です

In [8]:
payload = {
  "text": ["紅まどんな"],
  "tokenizer": {
    "type": "kuromoji_tokenizer",
    "mode": "search",
    "discard_compound_token": True 
  }
}
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,紅,0,1,word,0
1,ま,1,2,word,1
2,どんな,2,5,word,2


分割された各トークンの読みガナを確認すると、**アカ/マ/ドンナ** となっていることが確認できました。

In [9]:
payload = {
  "text": "紅まどんな",
  "tokenizer": {
    "type": "kuromoji_tokenizer",
    "mode": "search",
    "discard_compound_token": True 
  },
  "filter": [
    {
      "type": "kuromoji_readingform",
      "use_romaji": False
    }
  ]
}
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,アカ,0,1,word,0
1,マ,1,2,word,1
2,ドンナ,2,5,word,2


### ユーザー辞書の書式

Kuromoji はユーザー辞書として以下のフォーマットをサポートしています。

`<文字列>,<トークン 1> ... <トークン n>,<読みガナ 1> ... <読みガナ n>,<品詞タグ>`

1 つ目のエントリ**<文字列>**では処理対象の文字列を、2 つめのエントリ **<トークン 1> ... <トークン n>** では、入力された文字列の分割単位を、3 つめのエントリ **<読みガナ 1> ... <読みガナ n>** には、トークンの読みガナを、最後のエントリには品詞名を表すタグを記載します。品詞タグには`カスタム名詞`を用いるのが一般的です。

**紅まどんな** を **紅まどんな** のまま分割せずにトークン化したい場合は、以下のように記載します。

`紅まどんな,紅まどんな,ベニマドンナ,カスタム名詞`

このエントリを user_dictionary_rules に追加して、改めて _analyze API を実行し、"紅まどんな" が単体のトークンとして抽出されたことを確認します。

In [10]:
payload = {
  "text": "紅まどんな",
  "tokenizer": {
    "type": "kuromoji_tokenizer",
    "mode": "search",
    "discard_compound_token": True,
    "user_dictionary_rules": ["紅まどんな,紅まどんな,ベニマドンナ,カスタム名詞"]
  }
}
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,紅まどんな,0,5,word,0


kuromoji_readingform フィルタを追加して、トークンの読みガナも正しく処理されていることを確認します。

In [11]:
payload = {
  "text": "紅まどんな",
  "tokenizer": {
    "type": "kuromoji_tokenizer",
    "mode": "search",
    "discard_compound_token": True,
    "user_dictionary_rules": ["紅まどんな,紅まどんな,ベニマドンナ,カスタム名詞"]
  },
  "filter": [
    {
      "type": "kuromoji_readingform",
      "use_romaji": False
    }
  ]
}
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,ベニマドンナ,0,5,word,0


ユーザー辞書を活用することで、以下のようなトークン分割の調整を行うこともできます。

- **東京ゲートブリッジ** のように、デフォルトの挙動だと **東京/ゲート/ブリッジ** と 3 つに分割されてしまうトークンを **東京/ゲートブリッジ** と分割位置を調整する
- **アイストールラテ** のように単体のトークンとして認識されるものを アイス/トール/ラテ と分割する

以下はユーザー辞書追加前のトークン分割結果です

In [12]:
payload = {
  "text": ["東京ゲートブリッジ", "アイストールラテ"],
  "tokenizer": {
    "type": "kuromoji_tokenizer",
    "mode": "search",
    "discard_compound_token": True,
    "user_dictionary_rules": ["紅まどんな,紅まどんな,ベニマドンナ,カスタム名詞"]
  }
}
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,東京,0,2,word,0
1,ゲート,2,5,word,1
2,ブリッジ,5,9,word,2
3,アイストールラテ,10,18,word,103


以下はユーザー辞書定義追加後のトークン分割結果です。

In [13]:
payload = {
  "text": ["東京ゲートブリッジ", "アイストールラテ"],
  "tokenizer": {
    "type": "kuromoji_tokenizer",
    "mode": "search",
    "discard_compound_token": True,
    "user_dictionary_rules": [
      "紅まどんな,紅まどんな,ベニマドンナ,カスタム名詞",
      "東京ゲートブリッジ,東京 ゲートブリッジ,トウキョウ ゲートブリッジ,カスタム名詞",
      "アイストールラテ,アイス トール ラテ,アイス トール ラテ,カスタム名詞"
    ]
  }
}
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,東京,0,2,word,0
1,ゲートブリッジ,2,9,word,1
2,アイス,10,13,word,102
3,トール,13,16,word,103
4,ラテ,16,18,word,104


## ユーザー辞書の適用
実際のインデックスにユーザー辞書をセットし、辞書の有無による検索精度を比較していきます。Amazon OpenSearch Service では、Kuromoji において以下 2 通りのユーザー辞書セット方法を提供しています。

- user_dictionary_rules オプションに直接ユーザー辞書エントリを定義
- Amazon OpenSearch Service 独自の、カスタムパッケージ機能の利用

本ラボでは user_dictionary_rules オプションを使用してユーザー辞書をインデックスにセットしていきます。

### user_dictionary_rules オプションによるユーザー辞書の適用
前述の Analyzer API で使用した user_dictionary_rules オプションは、インデックスに対して適用することが可能です。
インデックスに対して直接エントリをセットできるため、動作確認を素早く行うことが可能です。

インデックスの定義内にエントリを含む性質上、大量のエントリを管理する場合はカスタムパッケージを使用した方がよいでしょう。

以降、user_dictionary_rules オプションを使用したユーザー辞書の適用方法を解説していきます。

#### インデックスの作成
item フィールドおよび、サブフィールドの item.text、item.text_with_userdict フィールドを持つインデックスを定義します。

item フィールドは keyword フィールドであるため、完全一致検索で用います。一方で item.text フィールドは Kuromoji のデフォルト辞書のみを使用してトークン分割を行います。item.text_with_userdict フィールドにはユーザー辞書を追加しています。

OpenSearch は、子フィールドを定義することで、親フィールドに投入した値を元に子フィールドごとに個別のインデックスを生成し、検索に使用することができます。単一フィールドへのデータ投入で複数のインデックスを作成できるため、クライアント -　OpenSearch 間のペイロードサイズを削減することができるなど、いくつかの面でメリットがあります。

In [14]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"

user_dictionary_rules = [
    "紅まどんな,紅まどんな,ベニマドンナ,カスタム名詞",
    "東京ゲートブリッジ,東京 ゲートブリッジ,トウキョウ ゲートブリッジ,カスタム名詞",
    "アイストールラテ,アイス トール ラテ,アイス トール ラテ,カスタム名詞"
]

payload = {
  "mappings": {
    "properties": {
      "id": {"type": "keyword"},
      "item": {
        "type": "keyword",
        "fields": {
          "text": {
            "type": "text",
            "analyzer": "custom_kuromoji_analyzer",
          },
          "text_with_userdict": {
            "type": "text",
            "analyzer": "custom_kuromoji_analyzer_with_userdict",
          }
        }
      }
    }
  },
  "settings": {
    "index.number_of_shards": 1,
    "index.number_of_replicas": 0,
    "analysis": {
      "analyzer": {
        "custom_kuromoji_analyzer": {
          "tokenizer": "custom_kuromoji_tokenizer"
        },
        "custom_kuromoji_analyzer_with_userdict": {
          "tokenizer": "custom_kuromoji_tokenizer_with_userdict",
        }
      },
      "tokenizer": {
        "custom_kuromoji_tokenizer": {
          "type": "kuromoji_tokenizer",
          "mode": "search",
          "discard_compound_token": True
        },
        "custom_kuromoji_tokenizer_with_userdict": {
          "type": "kuromoji_tokenizer",
          "mode": "search",
          "discard_compound_token": True,
          "user_dictionary_rules": user_dictionary_rules
        }
      }
    }
  }
}

try:
    # 既に同名のインデックスが存在する場合、いったん削除を行う
    print("# delete index")
    response = opensearch_client.indices.delete(index=index_name)
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)

# インデックスの作成を行う
print("# create index")
response = opensearch_client.indices.create(index=index_name, body=payload)
print(json.dumps(response, indent=2))

# delete index
NotFoundError(404, 'index_not_found_exception', 'no such index [kuromoji-sample-with-user-dictionary-rules-v1]')
# create index
{
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "kuromoji-sample-with-user-dictionary-rules-v1"
}


#### テストデータの投入
テストデータを投入します。<br>正しいデータと、元の正しいデータを並べ替えた不正なデータを登録し、検索結果の確認に用います。

In [15]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"

payload = f"""
{{"index": {{"_index": "{index_name}", "_id": "1a"}}}}
{{"item": "紅まどんな"}}
{{"index": {{"_index": "{index_name}", "_id": "1b"}}}}
{{"item": "どんな紅ま"}}
{{"index": {{"_index": "{index_name}", "_id": "2a"}}}}
{{"item": "東京ゲートブリッジ"}}
{{"index": {{"_index": "{index_name}", "_id": "2b"}}}}
{{"item": "東京ブリッジゲート"}}
{{"index": {{"_index": "{index_name}", "_id": "3"}}}}
{{"item": "アイストールラテ"}}
"""

response = opensearch_client.bulk(payload, refresh=False)

print(json.dumps(response, indent=2))

{
  "took": 148,
  "errors": false,
  "items": [
    {
      "index": {
        "_index": "kuromoji-sample-with-user-dictionary-rules-v1",
        "_id": "1a",
        "_version": 1,
        "result": "created",
        "_shards": {
          "total": 0,
          "successful": 0,
          "failed": 0
        },
        "_seq_no": 0,
        "_primary_term": 0,
        "status": 201
      }
    },
    {
      "index": {
        "_index": "kuromoji-sample-with-user-dictionary-rules-v1",
        "_id": "1b",
        "_version": 1,
        "result": "created",
        "_shards": {
          "total": 0,
          "successful": 0,
          "failed": 0
        },
        "_seq_no": 0,
        "_primary_term": 0,
        "status": 201
      }
    },
    {
      "index": {
        "_index": "kuromoji-sample-with-user-dictionary-rules-v1",
        "_id": "2a",
        "_version": 1,
        "result": "created",
        "_shards": {
          "total": 0,
          "successful": 0,
          "f

#### 検索の実行
まず、`紅まどんな` で、item.text フィールドに対して検索を行います。

In [16]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "紅まどんな"

payload = {
  "query": {
    "match": {
      "item.text": {
        "query": query,
        "operator": "and"
      }
    }
  },
  "highlight": {
    "fields": {
      "*" : {}
    }
  },
  "_source": False,
  "fields": ["item.text"]
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.item.text,highlight.item.text
0,kuromoji-sample-with-user-dictionary-rules-v1,1b,0.863046,[どんな紅ま],[<em>どんな</em><em>紅</em><em>ま</em>]
1,kuromoji-sample-with-user-dictionary-rules-v1,1a,0.863046,[紅まどんな],[<em>紅</em><em>ま</em><em>どんな</em>]


上記の結果より、クエリテキストが **紅/ま/どんな** にトークン分割されてからマッチング処理が実行されているため、不要なデータもヒットしていることが確認できました。<br>
では、ユーザー辞書によってトークン分割が適正化されている **item.text_with_userdict** フィールドに対して同様のクエリを発行します。


In [17]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "紅まどんな"

payload = {
  "query": {
    "match": {
      "item.text_with_userdict": {
        "query": query,
        "operator": "and"
      }
    }
  },
  "highlight": {
    "fields": {
      "*" : {}
    }
  },
  "_source": False,
  "fields": ["item.text_with_userdict"]
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.item.text_with_userdict,highlight.item.text_with_userdict
0,kuromoji-sample-with-user-dictionary-rules-v1,1a,0.287682,[紅まどんな],[<em>紅まどんな</em>]


正しく **紅まどんな** だけがヒットしました。また、ハイライトを見ると **紅まどんな** 全体が一つのトークンとして処理されていることが分かります。<br>
同様に、**東京ゲートブリッジ** で item.text フィールドの検索を行うと、**東京ブリッジゲート** もヒットしてしまいました。これは **東京/ゲート/ブリッジ** と 3 つのトークンに分割されてしまっていることが理由です。

In [18]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "東京ゲートブリッジ"

payload = {
  "query": {
    "match": {
      "item.text": {
        "query": query,
        "operator": "and"
      }
    }
  },
  "highlight": {
    "fields": {
      "*" : {}
    }
  },
  "_source": False,
  "fields": ["item.text"]
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.item.text,highlight.item.text
0,kuromoji-sample-with-user-dictionary-rules-v1,2b,0.863046,[東京ブリッジゲート],[<em>東京</em><em>ブリッジ</em><em>ゲート</em>]
1,kuromoji-sample-with-user-dictionary-rules-v1,2a,0.863046,[東京ゲートブリッジ],[<em>東京</em><em>ゲート</em><em>ブリッジ</em>]


対策としては match_phrase の利用が考えられますが、この場合、クエリに **東京ゲートブリッジ** 以外も含まれているとそちらも順序判定の対象となってしまいます。

In [19]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "東京ゲートブリッジ"

payload = {
  "query": {
    "match_phrase": {
      "item.text": {
        "query": query,
      }
    }
  },
  "highlight": {
    "fields": {
      "*" : {}
    }
  },
  "_source": False,
  "fields": ["item.text"]
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.item.text,highlight.item.text
0,kuromoji-sample-with-user-dictionary-rules-v1,2a,0.863046,[東京ゲートブリッジ],[<em>東京</em><em>ゲート</em><em>ブリッジ</em>]


ユーザー辞書によってトークン分割が適正化されている **item.text_with_userdict** フィールドに対して同様のクエリを発行します。

In [20]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "東京ゲートブリッジ"

payload = {
  "query": {
    "match": {
      "item.text_with_userdict": {
        "query": query,
        "operator": "and"
      }
    }
  },
  "highlight": {
    "fields": {
      "*" : {}
    }
  },
  "_source": False,
  "fields": ["item.text_with_userdict"]
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.item.text_with_userdict,highlight.item.text_with_userdict
0,kuromoji-sample-with-user-dictionary-rules-v1,2a,0.575364,[東京ゲートブリッジ],[<em>東京</em><em>ゲートブリッジ</em>]


**ゲートブリッジ** がトークン分割されないことで、正しいドキュメントだけを取得することができました。
最後にアイストールラテを検索していきます。アイストールラテを検索する際に、ラテのアイスでサイズはトール、と考えて **ラテ アイス トール** で検索を行います。

検索対象のフィールドは、ユーザー辞書が適用されていない **item.text** フィールドです。

In [21]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "ラテ アイス トール"

payload = {
  "query": {
    "match": {
      "item.text": {
        "query": query,
        "operator": "and"
      }
    }
  },
  "highlight": {
    "fields": {
      "*" : {}
    }
  },
  "_source": False,
  "fields": ["item.text"]
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

""


残念ながらヒットしません。これは **アイストールラテ** で単一トークンと認識されているためです。実際にアイストールラテで検索した結果は以下の通りです。

In [22]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "アイストールラテ"

payload = {
  "query": {
    "match": {
      "item.text": {
        "query": query,
        "operator": "and"
      }
    }
  },
  "highlight": {
    "fields": {
      "*" : {}
    }
  },
  "_source": False,
  "fields": ["item.text"]
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.item.text,highlight.item.text
0,kuromoji-sample-with-user-dictionary-rules-v1,3,0.287682,[アイストールラテ],[<em>アイストールラテ</em>]


ユーザー辞書によってトークン分割が適正化されている **item.text_with_userdict** フィールドに対して同様のクエリを発行します。

In [23]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "ラテ アイス トール"

payload = {
  "query": {
    "match": {
      "item.text_with_userdict": {
        "query": query,
        "operator": "and",
      }
    }
  },
  "highlight": {
    "fields": {
      "*" : {}
    }
  },
  "_source": False,
  "fields": ["item.text_with_userdict"]
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.item.text_with_userdict,highlight.item.text_with_userdict
0,kuromoji-sample-with-user-dictionary-rules-v1,3,0.863046,[アイストールラテ],[<em>アイス</em><em>トール</em><em>ラテ</em>]


無事にヒットしました。アイストールラテが **アイス/トール/ラテ** の 3 つのトークンに分割されることで、順序が異なるキーワードによる検索でもヒットするようになりました。

## ユーザー辞書の更新
ユーザー辞書は、検索要件やトレンドの変化に合わせて継続的なメンテナンスが必要です。Kuromoji のユーザー辞書更新はオンラインでインデックスに反映されないため、一般的には何らかの静止点を設けて更新作業を行う必要があります。 

辞書の更新を行う場合、一般的には以下 3 通りの作業方法から選択します。

### 既存のインデックスに格納されたデータを利用して、登録されたデータの更新を実施
既存のインデックスに格納されたソースデータを利用して、インデックス内のデータの再登録を行う方式です。[Update by query][update-by-query] API を実行するだけで再登録ができるため最も楽に更新を実行できますが、以下の点に注意する必要があります。

- 辞書定義の更新処理のために、一時的にインデックスを [Close index][close-index] API で閉じる必要あり。この間はデータ更新も検索もできない
- データの再登録処理中は、辞書更新前/更新後のデータが混在する。この間の検索結果は一貫性が保てない
- 辞書の更新による問題が発生した場合は、更新前のエントリに戻して再度 update_by_query を実行する必要がある
- 大量のドキュメントが登録されているインデックスに対する update_by_query は時間がかかる

作業は以下の流れで行います

1. (カスタムパッケージを利用している場合は)カスタムパッケージを更新
1. インデックスを [Close index][close-index] API で閉じる
1. (user_dictionary_rules) を使用している場合は、ここでユーザー辞書のエントリを更新
1. インデックスを [Open index][open-index] API で開ける
1. [Update by query][update-by-query] API によるドキュメントの再登録を実行


### 更新後の辞書が適用された新規インデックスを作成し、データを再登録
新しい辞書定義を含む空の新規インデックスを作成し、既存インデックスからデータをコピー、テスト後にトラフィックを新規のインデックスに切り替える方式です。

[Update by query][update-by-query] API 方式はインデックスの close を伴うため、検索処理も一時的にストップします。一方でこちらの方式は、データ更新こそ停止断面を確保する必要がありますが、検索処理を止めずに辞書の切り替えが可能です。このため、多くの本番運用で採用されています。

インデックス内のドキュメント更新処理を停止できることが理想です。ドキュメント更新処理を停止できない場合は、両系更新を検討するとよいです。

作業は以下の流れで行います。

1. ユーザー辞書のエントリを更新
1. 新しい辞書エントリを元に、新規にインデックスを作成
1. インデックスに対する更新処理を停止
1. 新規のインデックスにデータを再登録
1. [Alias][alias] API を使用し、現行インデックスから新規インデックスにエイリアスを切り替え
1. インデックスに対する更新処理を再開。以降は新規インデックスに対してデータ更新を行う

再登録は、初期登録時と同様に、マスターデータを外部から取得して Bulk API 等で書き込む方法と、[Reindex][reindex-data] API を使用する方法があります。Reindex API は、OpenSearch のインデックスに登録されたドキュメントを取得し、別のインデックスに書き込む機能です。

### インデックスの両系更新
更新処理に伴うデータ登録の停止時間が取れない場合は、両系更新を検討することになります。
ひとつ前のデータ再登録方式と似ていますが、既存インデックス用と新規インデックス用で、別々のデータ更新パイプライン(あるいはバッチ処理)を用意する必要がある点が異なります。

1. ユーザー辞書のエントリを更新
1. 新しい辞書エントリを元に、新規にインデックスを作成
1. 既存のデータ更新パイプラインと同じ構成のパイプラインを、新規インデックス向けにも構築し、既存インデックスと新規インデックスそれぞれで、データが最新状態を保てる状態を確保する
1. _alias API を使用し、現行インデックスから新規インデックスにエイリアスを切り替え
1. インデックスに対する更新処理を再開。以降は新規インデックスに対してデータ更新を行う

本ラボでは、update_by_query、および reindex + alias による辞書更新方法を解説します。

[update-by-query]: https://opensearch.org/docs/latest/api-reference/document-apis/update-by-query/
[open-index]: https://opensearch.org/docs/latest/api-reference/index-apis/open-index/
[close-index]: https://opensearch.org/docs/latest/api-reference/index-apis/close-index/
[alias]: https://opensearch.org/docs/latest/api-reference/index-apis/alias/
[reindex-data]: https://opensearch.org/docs/latest/im-plugin/reindex-data/

### user_dictionary_rules オプションを使用したインデックスに対する辞書更新
今度は、ホットミルクティーとホットミルクラテの区切り位置を改善することで、さらに検索精度を上げていきましょう。

#### デフォルトのトークン分割結果の確認
デフォルトの Kuromoji analyzer の挙動を見てみましょう。ホットミルクティーは **ホッ/トミルクティー** と分割されています。

In [24]:
payload = {
  "text": ["ホットミルクティー"],
  "tokenizer": {
    "type": "kuromoji_tokenizer",
    "mode": "search",
    "discard_compound_token": True 
  }
}
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,ホッ,0,2,word,0
1,トミルクティー,2,9,word,1


同様にホットミルクラテも **ホッ/トミルクラテ** に分割されます。

In [25]:
payload = {
  "text": ["ホットミルクラテ"],
  "tokenizer": {
    "type": "kuromoji_tokenizer",
    "mode": "search",
    "discard_compound_token": True
  }
}
response = opensearch_client.indices.analyze(
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,ホッ,0,2,word,0
1,トミルクラテ,2,8,word,1


実際にホットミルクティーを登録して検索を行ってみましょう

In [26]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"

payload = f"""
{{"index": {{"_index": "{index_name}", "_id": "4"}}}}
{{"item": "ホットミルクティー"}}
"""

response = opensearch_client.bulk(payload, refresh=False)
print(json.dumps(response, indent=2))

{
  "took": 112,
  "errors": false,
  "items": [
    {
      "index": {
        "_index": "kuromoji-sample-with-user-dictionary-rules-v1",
        "_id": "4",
        "_version": 1,
        "result": "created",
        "_shards": {
          "total": 0,
          "successful": 0,
          "failed": 0
        },
        "_seq_no": 0,
        "_primary_term": 0,
        "status": 201
      }
    }
  ]
}


**ホットミルクティー** では item.text フィールドおよび item.text_with_userdict フィールド双方にヒットします。

In [27]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "ホットミルクティー"

payload = {
  "query": {
    "multi_match": {
      "fields": ["item.text", "item.text_with_userdict"],
      "query": query,
      "operator": "and"
    }
  },
  "highlight": {
    "fields": {
      "*" : {}
    }
  },
  "_source": False,
  "fields": ["item.text", "item.text_with_userdict"]
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.item.text,fields.item.text_with_userdict,highlight.item.text,highlight.item.text_with_userdict
0,kuromoji-sample-with-user-dictionary-rules-v1,4,1.509825,[ホットミルクティー],[ホットミルクティー],[<em>ホッ</em><em>トミルクティー</em>],[<em>ホッ</em><em>トミルクティー</em>]


**ミルクティー ホット** ではいずれのフィールドにもヒットしません。ホットミルクティーに対するユーザ辞書エントリがないので、これは想定通りの結果といえます。

In [28]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "ミルクティー ホット"

payload = {
  "query": {
    "multi_match": {
      "fields": ["item.text", "item.text_with_userdict"],
      "query": query,
      "operator": "and"
    }
  },
  "highlight": {
    "fields": {
      "*" : {}
    }
  },
  "_source": False,
  "fields": ["item.text", "item.text_with_userdict"]
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

""


ここで、ヒットしない理由を API を使って確かめていきましょう。

[Analyze][analyze] API と [Explain][explain] API を実行して、登録時と検索時のトークン分割の様子を比較していきます。

まずは、Analyze API を実行して、**ホットミルクティー** を登録する際に、文字列がどのようにトークン分割されているかを確認します。

[analyze]: https://opensearch.org/docs/latest/api-reference/analyze-apis/
[explain]: https://opensearch.org/docs/latest/api-reference/explain/

In [29]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"

payload = {
  "text": "ホットミルクティー",
  "analyzer": "custom_kuromoji_analyzer_with_userdict"
}
response = opensearch_client.indices.analyze(
  index=index_name,
  body=payload
)
pd.json_normalize(response["tokens"])

,token,start_offset,end_offset,type,position
0,ホッ,0,2,word,0
1,トミルクティー,2,9,word,1


次に、Explain API を利用することで、クエリテキストがどのように分解されて内部で検索処理が行われているかを確認します。Analyzer API にクエリテキストを渡しても確認することができますが、Explain API ではクエリと対象のドキュメントを指定することで、実際に分割後のトークンごとにマッチするか確認できます。

In [30]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "ホットミルクティー"

payload = {
  "query": {
    "match": {
      "item.text": {
        "query": query,
        "operator": "and"
      }
    }
  },
}
response = opensearch_client.explain(
    index=index_name,
    body=payload,
    id=4 #ドキュメント"ホットミルクティー"の ID
)
print(json.dumps(response, indent=2, ensure_ascii=False))

{
  "_index": "kuromoji-sample-with-user-dictionary-rules-v1",
  "_id": "4",
  "matched": true,
  "explanation": {
    "value": 1.2199391,
    "description": "sum of:",
    "details": [
      {
        "value": 0.60996956,
        "description": "weight(item.text:ホッ in 0) [PerFieldSimilarity], result of:",
        "details": [
          {
            "value": 0.60996956,
            "description": "score(freq=1.0), computed as boost * idf * tf from:",
            "details": [
              {
                "value": 2.2,
                "description": "boost",
                "details": []
              },
              {
                "value": 0.6931472,
                "description": "idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:",
                "details": [
                  {
                    "value": 1,
                    "description": "n, number of documents containing term",
                    "details": []
                  },
                  {
         

ドキュメント登録時のトークンと、クエリ時のトークン、いずれも **ホッ/トミルクティー**であるため、検索ヒットしたことが確認できました。

では、**ミルクティー ホット** ではどうでしょうか。

In [31]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "ミルクティー ホット"

payload = {
  "query": {
    "match": {
      "item.text": {
        "query": query,
        "operator": "and"
      }
    }
  },
}
response = opensearch_client.explain(
    index=index_name,
    body=payload,
    id=4 #ドキュメント"ホットミルクティー"の ID
)
print(json.dumps(response, indent=2, ensure_ascii=False))

{
  "_index": "kuromoji-sample-with-user-dictionary-rules-v1",
  "_id": "4",
  "matched": false,
  "explanation": {
    "value": 0.0,
    "description": "Failure to meet condition(s) of required/prohibited clause(s)",
    "details": [
      {
        "value": 0.0,
        "description": "no match on required clause (item.text:ミルク)",
        "details": [
          {
            "value": 0.0,
            "description": "no matching term",
            "details": []
          }
        ]
      },
      {
        "value": 0.0,
        "description": "no match on required clause (item.text:ティー)",
        "details": [
          {
            "value": 0.0,
            "description": "no matching term",
            "details": []
          }
        ]
      },
      {
        "value": 0.0,
        "description": "no match on required clause (item.text:ホット)",
        "details": [
          {
            "value": 0.0,
            "description": "no matching term",
            "details": []
       

**ミルクティー ホット** で検索を行った場合、クエリは **ミルク/ティー/ホット** にトークン分割されていることが分かります。

一方、Analyze API 実行結果より、**ホットミルクティー** というドキュメントは、登録時に **ホッ/トミルクティー** というトークンに分割されていることが確認できています。

検索時のトークンと、登録時のトークンにずれがあることが、**ミルクティー ホット** で **ホットミルクティー**が検索できない原因であると確認できました。

登録されている時と同じトークン分割結果である **ホッ/トミルクティー** で検索してみましょう。このクエリは一見すると不自然ですが、登録時のトークンと同一であることからヒットします。

In [32]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "ホッ トミルクティー"

payload = {
  "query": {
    "match": {
      "item.text": {
        "query": query,
        "operator": "and"
      }
    }
  },
  "highlight": {
    "fields": {
      "*" : {}
    }
  },
  "_source": False,
  "fields": ["item.text"]
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.item.text,highlight.item.text
0,kuromoji-sample-with-user-dictionary-rules-v1,4,1.219939,[ホットミルクティー],[<em>ホッ</em><em>トミルクティー</em>]


また、**ホッ** や **トミルクティー** など、単一のトークンで検索してもヒットしてしまいます。

In [33]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "ホッ"

payload = {
  "query": {
    "match": {
      "item.text": {
        "query": query,
        "operator": "and"
      }
    }
  },
  "highlight": {
    "fields": {
      "*" : {}
    }
  },
  "_source": False,
  "fields": ["item.text"]
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.item.text,highlight.item.text
0,kuromoji-sample-with-user-dictionary-rules-v1,4,0.60997,[ホットミルクティー],[<em>ホッ</em>トミルクティー]


In [34]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"
query = "トミルクティー"

payload = {
  "query": {
    "match": {
      "item.text": {
        "query": query,
        "operator": "and"
      }
    }
  },
  "highlight": {
    "fields": {
      "*" : {}
    }
  },
  "_source": False,
  "fields": ["item.text"]
}
response = opensearch_client.search(
    index=index_name,
    body=payload
)
pd.json_normalize(response["hits"]["hits"])

,_index,_id,_score,fields.item.text,highlight.item.text
0,kuromoji-sample-with-user-dictionary-rules-v1,4,0.60997,[ホットミルクティー],[ホッ<em>トミルクティー</em>]


#### ユーザー辞書定義の更新
analyze API を使用することで入力時のトークン分割の様子が、explain API を使用することで検索時のトークン分割の様子が分かりました。登録・検索時のトークン分割を一致させることが、検索精度向上の鍵であることも分かりました。

ここからは、**ミルクティー ホット** でもヒットするように、**ホットミルクティー** が **ホット/ミルク/ティー** で区切られるようにユーザー辞書を作成していきましょう。

既存インデックスの user_dictonary_rules を以下の通り更新します。
合わせて **ホットミルクラテ** 用のエントリーも追加しておきます。

更新作業の前後で、Close API によるインデックスのクローズ、Open API によるインデックスのオープンを実行しています。

<div class="alert alert-block alert-warning"> 
本ラボで使用しているインデックスはサイズが小さいため、Close から Open も含む更新にかかる所要時間は 1 秒未満ですが、一般的に Close/Open にかかる時間はインデックスサイズに応じて増加していくため、本番環境で本作業を実施する場合は事前の検証が必要です。
</div>

> AOSS では、`close()` がサポートされていないので、スキップします。

#### Update by query API によるデータの再登録

既存のドキュメントを、新しい辞書エントリを元に改めてトークン分割しなおすためには、ドキュメントの再登録が必要となります。

外部にマスターデータがある場合は、外部から改めてデータの全登録を行うことがお勧めですが、本セクションでは [Update by query][update-by-query] API を使用します。Update by query API を実行することで、インデックスに登録されたドキュメント自身のデータをもとに、ドキュメントの再登録を行うことができます。

Update by query は完了まで長時間要する場合があるため、wait_for_completion オプションに False をセットし、非同期で実行することを推奨します。

[update-by-query]: https://opensearch.org/docs/latest/api-reference/document-apis/update-by-query/

> AOSS では、`update_by_query()` も使えません。

#### Reindex API によるデータの再登録 + Alias API によるアクセス先インデックスの切り替え
ここからは、Reindex API と Alias API を組み合わせた辞書更新について解説します。

##### エイリアスの登録
エイリアスは、インデックスに付与可能な別名です。

エイリアスはインデックス間でオンラインでの付け替えが可能であるため、バージョンが複数存在するインデックスに対して、クライアントからは常に同じ名前でアクセスしたい場合に有用です。

エイリアスは、Alias API を使用して付与します。以下のサンプルコードでは、インデックス **kuromoji-sample-with-user-dictionary-rules-v1** にエイリアス **kuromoji-sample-with-user-dictionary-rules** をセットしています。

> AOSS では、`put_alias()` が使えません。

##### Reindex による部分更新
Reindex 実行時にクエリパラメーターを追加することで、特定の条件に合致したドキュメントだけをコピーすることができます。クエリで更新対象のドキュメントの絞り込みが可能である場合は、Reindex 実行時間を短縮することが可能です。

例えば、更新時刻を示すフィールドを持っているドキュメントであれば、range クエリで特定時刻以前のドキュメントのみ再登録を行うことが可能です。

今回は、v1 にドキュメントが追加されたことを想定して、差分コピーを実行していきます。

> AOSS では、`reindex()` もできません。

## まとめ
本ラボでは、OpenSearch の日本語検索について学習しました。<br>
また、Kuromoji のユーザ辞書カスタマイズによる日本語検索の精度改善について学習しました。

本ラボで学習した内容を元に、次のステップとして以下のラボを実行してみましょう。

### ベクトル検索など他の検索手法を学びたい方向け
- [ベクトル検索の実装 (Amazon Bedrock 編)](4-ai-search.ipynb)

## 後片付け

### インデックス削除
本ワークショップで使用したインデックスを削除します。インデックスの削除は Delete index API で行います。インデックスを削除するとインデックス内のドキュメントも削除されます。

In [35]:
index_name = "jsquad-kuromoji"

try:
    response = opensearch_client.indices.delete(index=index_name)
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)

{
  "acknowledged": true
}


In [36]:
index_name = "kuromoji-sample-with-user-dictionary-rules-v1"

try:
    response = opensearch_client.indices.delete(index=index_name)
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)

{
  "acknowledged": true
}


### データセット削除
ダウンロードしたデータセットを削除します。./dataset ディレクトリ配下に何もない場合は、./dataset ディレクトリも合わせて削除します。

In [37]:
%rm -rf {dataset_dir}

In [38]:
%rmdir ./dataset

rmdir: failed to remove './dataset': Directory not empty
